<a href="https://colab.research.google.com/github/deepakgowtham/EVA4/blob/master/Week14/Session15A_Mask_RCNN_stat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import os
from skimage import io
from tqdm import tqdm


Mean/STD values for your fg_bg, masks and depth images

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [0]:
!mkdir /content/Mask_RCNN/fg_bg_mask/fg_bg_mask
!mkdir /content/Mask_RCNN/Depth_output/Depth_output

In [0]:
import shutil

source='/content/Mask_RCNN/fg_bg_mask/'
dest1= '/content/Mask_RCNN/fg_bg_mask/fg_bg_mask'
files = os.listdir(source)

for f in tqdm(files):
    shutil.move(source+f, dest1)

100%|██████████| 424321/424321 [00:35<00:00, 11814.61it/s]


In [0]:
len(os.listdir(dest1))

424320

In [0]:
torch.cuda.is_available()

True

In [0]:
transform = transforms.Compose([transforms.ToTensor()])
dataset = datasets.ImageFolder(root='/content/Mask_RCNN/fg_bg/', transform=transform)


loader = DataLoader(
    dataset,
    batch_size=600,
    num_workers=1,
    shuffle=False,
    pin_memory=True
)


mean = 0.
std = 0.
nb_samples = 0.
for data,_ in tqdm(loader):
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples

100%|██████████| 708/708 [21:21<00:00,  1.81s/it]


In [0]:
mean, std

(tensor([0.3931, 0.3785, 0.3606]), tensor([0.1965, 0.1813, 0.1779]))

In [0]:
lis1=os.listdir('/content/Mask_RCNN/Depth_output/content/Mask_RCNN/Depth_output')
lis1[:5]

['bg_60fg_63_flip_20.jpg',
 'bg_83fg_9_10.jpg',
 'bg_60fg_36_4.jpg',
 'bg_8fg_45_13.jpg',
 'bg_3fg_51_flip_3.jpg']

In [0]:
!cd /content/Mask_RCNN/mask; unzip /content/gdrive/My\ Drive/RCNN/fg_bg_mask.zip 

In [0]:
transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),transforms.ToTensor()])
dataset = datasets.ImageFolder(root='/content/Mask_RCNN/mask/Mask_RCNN', transform=transform)


loader = DataLoader(
    dataset,
    batch_size=512,
    num_workers=1,
    shuffle=False,
    pin_memory=True
)


mask_mean = 0.
mask_std = 0.
nb_samples = 0.
for data,_ in tqdm(loader):
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mask_mean += data.mean(2).sum(0)
    mask_std += data.std(2).sum(0)
    nb_samples += batch_samples

mask_mean /= nb_samples
mask_std /= nb_samples

mask_mean, mask_std

100%|██████████| 829/829 [05:04<00:00,  2.73it/s]


(tensor([0.1630]), tensor([0.3598]))

In [0]:
!cd /content/Mask_RCNN/depth; unzip /content/gdrive/My\ Drive/RCNN/Depth_output.zip

In [0]:
transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),transforms.ToTensor()])
dataset = datasets.ImageFolder(root='/content/Mask_RCNN/depth/content/Mask_RCNN', transform=transform)


loader = DataLoader(
    dataset,
    batch_size=512,
    num_workers=1,
    shuffle=False,
    pin_memory=True
)


depth_mean = 0.
depth_std = 0.
nb_samples = 0.
for data,_ in tqdm(loader):
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    depth_mean += data.mean(2).sum(0)
    depth_std += data.std(2).sum(0)
    nb_samples += batch_samples

depth_mean /= nb_samples
depth_std /= nb_samples

depth_mean, depth_std

100%|██████████| 829/829 [05:00<00:00,  2.76it/s]


(tensor([0.0878]), tensor([0.0157]))